In [1]:
# DOWNLOADS and IMPORTS
!pip install anvil-uplink
!pip install cloudscraper

import re
import requests
import string
import cloudscraper
import numpy as np
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from google.colab import drive
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import os.path
from os import path

# ANVIL SERVER SETUP
import anvil.server
anvil.server.connect("QRYI6F3U3MUJFO4J7F27WAU7-WH56GKJFAI26ZPDJ")

# NLP library
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')


     |████████████████████████████████| 61kB 3.1MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-cp37-none-any.whl size=45216 sha256=997b239edda871fd8e8545439e03b3fddf42a590d5c066231cf67535491fbbe5
  Stored in directory: /root/.cache/pip/wheels/a2/6e/4e/8b0ae12fb9b8a05715256952cf7609a8ab86285fab99b88c68
Successfully built ws4py


     |████████████████████████████████| 102kB 4.1MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 
Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.


True

In [ ]:
drive.mount('/content/drive')
user = "shinjini" #anil, vibha, shinjini, stuti

dat_file = 'Data_Bigger.csv'
if user == "vibha": 
  filepath = '/content/drive/MyDrive/MITDocuments/6.871 MLHC Project/'
elif user == "shinjini":
  filepath = '/content/'
else:
  filepath = '/content/drive/MyDrive/'

df = pd.read_csv(filepath + dat_file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def get_meds_from_row(i, with_dose=False):
  """ Given: a row number and an optional argument whether we want the medications
  with dosage information
  Returns: a set of medications given to the patient in corresponding row"""
  meds = set()
  
  med_starts = ['discharge medications:', 'medications on discharge:']
  med_start_idx = -1
  j = 0
  while med_start_idx < 0 and j < len(med_starts):
    med_start_idx = df.text[i].lower().find(med_starts[j])
    if med_start_idx > 0:
      med_start_idx = med_start_idx + len(med_starts[j])
    j = j + 1
  
  med_ends = ['discharge', 'follow up', '[**']
  med_end_idx = -1
  j = 0
  while med_end_idx < 0 and j < len(med_ends):
    med_end_idx = df.text[i][med_start_idx:].lower().find(med_ends[j])
    j = j + 1

  med_text = df.text[i][med_start_idx:med_start_idx+med_end_idx]
  meds_list_unprocessed = med_text.strip().split("\n")
  for med_description in meds_list_unprocessed:
    try:
      med_name = re.search('\d\. (.+?) (\d+\.?\d*|one|two) (m.*?(g|q)|unit|puff)', med_description).group(1)
      meds.add(med_name)
    except:
      try:
        med_name = re.search('(.+?) (\d+\.?\d*|one|two) (m.*?(g|q)|unit|puff)', med_description).group(1)
        meds.add(med_name)
      except:
        continue
  return meds

def get_conds_from_row(i):
  """ Given: a row number
  Returns: a set of patient conditions for corresponding row"""
  conds = set()
  
  cond_starts = ['discharge diagnosis:', 'final diagnoses:', 'discharge diagnoses:', 'final diagnosis:']
  cond_start_idx = -1
  j = 0
  while cond_start_idx < 0 and j < len(cond_starts):
    cond_start_idx = df.text[i].lower().find(cond_starts[j])
    if cond_start_idx > 0:
      cond_start_idx = cond_start_idx + len(cond_starts[j])
    j += 1
  
  cond_ends = ['discharge', 'follow up', '[**']
  cond_end_idx = -1
  j = 0
  while cond_end_idx < 0 and j < len(cond_ends):
    cond_end_idx = df.text[i][cond_start_idx:].lower().find(cond_ends[j])
    j += 1

  cond_text = df.text[i][cond_start_idx:cond_start_idx+cond_end_idx]
  cond_list_unprocessed = cond_text.strip().split("\n")
  #print(cond_list_unprocessed)
  for cond_description in cond_list_unprocessed:
    try:
      cond_name = re.search('\d\.\s*(.+?)(\.|,|$)', cond_description).group(1)
      conds.add(cond_name)
    except:
      try:
        cond_name = re.search('(.+?)\s*(\.|,|\[|$)', cond_description).group(1)
        conds.add(cond_name)
      except:
        continue
  return conds

In [ ]:
def get_google_def(med):
  """Given: a medication name
  Returns: drug name, brand name, description, typical and severe side effects from webmd"""
  #print("Searching google/webmd")
  drug_name = ""
  brand_names = []
  desc = "Not found" 
  ts = []
  ss = []
  i = 0
  url1 = "https://www.google.com/search?q=" + med + "+webmd"
  scraper = cloudscraper.create_scraper(delay = 15)
  my_text = scraper.get(url1).text
  soup = BeautifulSoup(my_text, "html.parser")
  
  links = soup.find_all('a')
  for l in links:
    try:
      if "https://www.webmd.com/drugs/2" in l.get('href'):
        k = l.get('href')
        myLink = k.split("details")[0] + "details"
        myLink = "https://" + k.split("https://")[1]
        break
    except:
      continue

  if myLink is None:
    return drug_name, brand_names, desc, ts, ss
  else:
    my_text = scraper.get(myLink).text
    soup = BeautifulSoup(my_text, "html.parser")
    try:
      desc = soup.find_all(class_="drug-uses")[-1].getText()
    except:
      desc = soup.find(class_="monograph-content").getText()
    desc = desc.encode("ascii", "ignore")

    try: 
      drug_name = soup.find(class_="drug-name").getText()
    except:
      drug_name = "Not found"
    
    try:
      bnames = soup.find_all(class_="drug-generic-name")
      for b in bnames:
        brand_names.append(b.getText())
    except:
      brand_names = brand_names
    return drug_name, brand_names, desc, ts, ss


def get_medline_description(med):
  """Given: a medication name
  Returns: drug name, brand name, description, typical and severe side effects from medline"""
  drug_name = ""
  brand_names = []
  desc = "Not found" 
  ts = []
  ss = []
  URL_1 = 'https://vsearch.nlm.nih.gov/vivisimo/cgi-bin/query-meta?v%3Aproject=medlineplus&v%3Asources=medlineplus-bundle&query=' + med + '&binning-state=group%3d%3dDrugs%20and%20Supplements&'
  scraper = cloudscraper.create_scraper(delay = 15)  
  my_text = scraper.get(URL_1).text
  soup = BeautifulSoup(my_text, "html.parser")

  #Going through the outputted urls in order, 
  #Picking the first one unless we deem it incorrect
  matches = soup.find_all(class_ = "url")
  names = soup.find_all(class_= "title")
  i = 0
  name = "Ravioli, ravioli, "
  URL_2 = "give me the formuoli"
  while (notCorrect(med, name, URL_2)):
    try:
      URL_2 = matches[i].text
    except:
      desc = "Not found"
      return drug_name, brand_names, desc, ts, ss
    name = names[i].text
    i = i + 1
  # Going to the drug's page
  try:
    scraper = cloudscraper.create_scraper(delay = 15)
    my_text_2 = scraper.get(URL_2).text
    soup2 = BeautifulSoup(my_text_2, "html.parser")
    matches_2 = soup2.find_all(id = "why")
    desc = matches_2[0].contents[1].text
    desc = desc.encode("ascii", "ignore")

    #Getting the drug name
    matches_3 = soup2.find(class_="with-also")
    drug_name = matches_3.text
    #Getting brand names for the drug
    try:
      matches_4 = soup2.find(id="section-brandname-1").contents[0]
    except:
      matches_4 = soup2.find(id="section-brand-name-2").contents[0]
    brand_names = []
    for li in matches_4.find_all('li'):
      brand_names.append(li.text)

    side_effect_matches = soup2.find(id = "side-effects")
    try:
      typical_sides = side_effect_matches.contents[1].contents[1]
      ts = []
      for li in typical_sides.find_all('li'):
        ts.append(li.text)
    except:
      ts = []
    
    try:
      severe_sides = side_effect_matches.contents[1].contents[3]
      ss = []
      for li in severe_sides.find_all('li'):
        ss.append(li.text)
      for li in severe_sides.parent.contents[2].find_all('li'):
        ss.append(li.text)
    except:
      ss = []
    return drug_name, brand_names, desc, ts, ss
  except:
    desc = "Not found"
  return drug_name, brand_names, desc, ts, ss

@anvil.server.callable
#Helper function for get_med_descriptions
def get_med_description(med, get_side_effect = False):

  '''
  Search medlineplus for a description of med
  Input: string
  Output: string
  '''
  med = str(med)
  med = med.lower()
  med = med.replace(" ", "+")

  drug_name = ""
  brand_names = []
  desc = "Not found" 
  ts = []
  ss = []

  try:
    drug_name, brand_names, desc, ts, ss = get_medline_description(med)
    if desc != "Not found" and med in desc.lower():
      return drug_name, brand_names, desc, ts, ss
  except:
    desc = "Not found"
  
  try:
    drug_name, brand_names, desc, ts, ss= get_google_def(med)
    if desc != "Not found":
      return drug_name, brand_names, desc, ts, ss
  except:
    desc = "Not found"
  
  return drug_name, brand_names, desc, ts, ss


#Helper function for selecting valid drug search URL
#Add cases as we test
def notCorrect(med, name, URL):
  """Given: a medicine, name and URL
  Returns: if the URL is the valid drug search URL for the given medication"""
  m = med.lower()
  n = name.lower()
  if name + URL == "Ravioli, ravioli, give me the formuoli":
    return True
  if "injection" not in m and "injection" in n:
    return True
  elif " and " not in m and " and " in n:
    return True
  elif "druginfo" not in URL:
    return True
  else:
    return False
  

In [ ]:
def get_google_cond(cond):
  """Given: a condition name
  Returns: the webmd description for the condition 
  """
  desc = "Not found"
  try:
    url1 = "https://www.google.com/search?q=" + cond + "+webmd"
    scraper = cloudscraper.create_scraper(delay = 15)
    my_text = scraper.get(url1).text
    soup = BeautifulSoup(my_text, "html.parser")
    
    links = soup.find_all('a')
    for l in links:
      try:
        if "https://www.webmd.com/" in l.get('href'):
          k = l.get('href')
          myLink = "https://" + k.split("https://")[1]
          break
      except:
        continue
    
    if myLink is None:
      return desc
    else:
      my_text = scraper.get(myLink).text
      soup = BeautifulSoup(my_text, "html.parser")
      try:
        desc = soup.find(class_="article-page active-page").getText()
      except:
        return desc
      desc = desc.encode("ascii", "ignore")
    return desc
  except:
    return desc

def get_full_desc(condition):
  '''
  Search medlineplus for a description of condition
  Input: string
  Output: string
  '''
  try:
    cond = str(condition)
    cond = cond.replace(" ", "+")
    URL_1 = 'https://vsearch.nlm.nih.gov/vivisimo/cgi-bin/query-meta?v%3Aproject=medlineplus&v%3Asources=medlineplus-bundle&query=' + cond + '&binning-state=group%3d%3dHealth%20Topics&'
    scraper = cloudscraper.create_scraper(delay = 15)  
    my_text = scraper.get(URL_1).text
    soup = BeautifulSoup(my_text, "html.parser")

    #Going through the outputted urls in order, 
    #Picking the first one unless we deem it incorrect
    matches = soup.find_all(class_ = "url")
    names = soup.find_all(class_= "title")
    i = 0
    name = "Ravioli, ravioli, "
    URL_2 = "give me the formuoli"
    while (notCorrectCond(cond, name, URL_2)):
      try:
        URL_2 = matches[i].text
      except:
        desc = "Not found"
        return get_google_cond(cond)
      name = names[i].text
      i = i + 1

    scraper = cloudscraper.create_scraper(delay = 15)
    my_text_2 = scraper.get(URL_2).text
    soup2 = BeautifulSoup(my_text_2, "html.parser")

    #Getting the condition full description
    matches_2 = soup2.find_all(id = "topsum_section")
    matches_summary = matches_2[0]
    desc = matches_summary.getText()
    desc = desc.encode("ascii", "ignore")
    return desc
  except:
    desc = get_google_cond(cond)
    return desc


#Helper function for selecting valid drug search URL
#Add cases as we test
def notCorrectCond(cond, name, URL):
  m = cond.lower()
  n = name.lower()
  if name + URL == "Ravioli, ravioli, give me the formuoli":
    return True
  return False

In [ ]:
# Functions to get patient conditions and medications
@anvil.server.callable
def get_pt_conditions(hadm_id, df=df):
  """
  Input: a dataframe and a patient id
  Returns: a list of condition names
  """
  all_conds = set()
  for index, row in df.iterrows():
    if row.hadm_id == hadm_id:
      all_conds.update(get_conds_from_row(index))
  orig_cond_list = sorted(list(all_conds))
  final_conds_list = []
  for i, c in enumerate(orig_cond_list):
    con = get_full_desc(c)
    if con != "Not found":
      final_conds_list.append(c)

  return final_conds_list

@anvil.server.callable
def get_pt_medications(hadm_id, with_dose=False, df=df):
  """ 
  Given: a dataframe of patient information, hadm_id and an optional argument whether we want the medications
  with dosage information
  Returns: a list of medications, with dose details depending on parameter
  """
  all_meds = set()
  for index, row in df.iterrows():
    if row.hadm_id == hadm_id:
      all_meds.update(get_meds_from_row(index, with_dose=with_dose))
  
  orig_med_list = sorted(list(all_meds))
  final_med_list = []
  for i, m in enumerate(orig_med_list):
    d, b, mdesc, n, s = get_med_description(m)
    if mdesc != "Not found":
      final_med_list.append(m)
  return final_med_list

In [ ]:
# Functions for NLP training step
def get_corpus(paragraphs):
  """
  Input: list of strings
  Takes each string and strips it so that only nouns remain 
  Output: tuple, first item is a list of noun-only strings, 
    second is list of integers representing the length of the noun-only strings
  """
  nltk_types = ['NN', 'NNP', 'NNS', 'NNPS']  #nltk.help.upenn_tagset() to see docum.
  corpus = []
  lens = []
  for par in paragraphs:
    tokens = nltk.word_tokenize(par)
    tagged = nltk.pos_tag(tokens)
    nouns_only = ' '.join([word for word, typ in tagged if typ in nltk_types])
    corpus.append(nouns_only)
    lens.append(len(nouns_only.split()))
  lens = np.array(lens)[:, None]
  return corpus, lens

In [ ]:
corpus_pickle_location = filepath + 'corpus.pickle'

# Load re-saved corpuses
if path.exists(corpus_pickle_location):
  # load the saved corpuses
  with open(corpus_pickle_location, 'rb') as handle:
    corpus_conds, cond_lens, corpus_meds, med_lens, corpus_rands, rand_lens, corpus_conds_rand, corpus_meds_rand, vectorizer = pickle.load(handle)

# total number of noun-only sentences in each corpus
num_conds_total = len(corpus_conds)
num_meds_total = len(corpus_meds)
num_rands_total = len(corpus_rands)

# fit the vectorizer to the corpus
vectorizer = CountVectorizer(ngram_range= (1, 1))
corpus_full_nums = vectorizer.fit_transform(corpus_conds + corpus_meds)

# transform corpuses with vectorizer
cnarr_conds = vectorizer.transform(corpus_conds).toarray()
cnarr_meds = vectorizer.transform(corpus_meds).toarray()
cnarr_rands = vectorizer.transform(corpus_rands).toarray()

# percent of articles with at least 1 occurence
word_freqs_conds = np.sum((cnarr_conds > 0).astype(int), axis = 0)/num_conds_total
word_freqs_meds = np.sum((cnarr_meds > 0).astype(int), axis = 0)/num_meds_total
word_freqs_rands = np.sum((cnarr_rands > 0).astype(int), axis = 0)/num_rands_total

# avg times per word per article normalized by article len
avg_word_counts_conds = np.sum(cnarr_conds[:(num_conds_total)]/cond_lens, axis = 0)/(num_conds_total)
avg_word_counts_meds = np.sum(cnarr_meds[:(num_meds_total)]/med_lens, axis = 0)/(num_meds_total)
avg_word_counts_rands = np.sum(cnarr_rands[:(num_rands_total)]/rand_lens, axis = 0)/(num_rands_total)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
def isImportantWord(i, condvec, medvec, condlen, medlen, word_list):
  """
  Function to determine if a word is "important". 
  - it must appear infrequently across all articles
    ie "disease" and "treatment" are too common but "cholesterol" and "antidepressant" are more unique
  - it must be more common for the given condition+medication than the average condition+medication
    ie "blood pressure" might appear once or twice in most articles, but 4 times for hypertension and hypertension drugs
  """
  cond_freq = .25
  med_freq = .25
  rand_freq =.01 #.0015 #we are actually referring to true rand freq/cond freq
  
  # important word if the word appears in less than 1/nth of the cond/med articles (+ wiki descriptions)
  if word_freqs_conds[i] < cond_freq and word_freqs_meds[i] < med_freq and word_freqs_rands[i]/word_freqs_conds[i] < rand_freq: 
    # important word if the word appears more in the chosen cond on average than it does in the average condition and likewise for med
    if condvec[0, i]/condlen > 1.5*avg_word_counts_conds[i] and medvec[0, i]/medlen > 1.5*avg_word_counts_meds[i]:
      #print(word_list[i] + " " +str(np.round(word_freqs_conds[i], 4)) + " " + str(np.round(word_freqs_meds[i], 4)) + " " + str(np.round(word_freqs_rands[i], 4)))
      return True
  return False


def get_mutual_words(cond, med, cond_descr, med_descr):
  """
  Finds (noun-only) words that are in both articles and are "important" (see helper function). 
  Selects the words that have highest frequency and outputs them as the keywords
  """
  # preprocess in the same way we did to the corpus
  cond_descr, clen = get_corpus([cond_descr])
  med_descr, mlen = get_corpus([med_descr])
  # cond_descr, med_descr, clen, mlen all are lists of len 1
  
  cond_vec = vectorizer.transform(cond_descr)
  med_vec = vectorizer.transform(med_descr)
  word_list = vectorizer.get_feature_names()
  
  # how many of each word in the paragraph (1d vectors)
  cond_word_count = cond_vec.toarray().sum(axis=0)
  med_word_count = med_vec.toarray().sum(axis=0)

  words_in_both_idx = np.logical_and(cond_word_count > 0, med_word_count > 0)
  both_idx = np.nonzero(words_in_both_idx)[0]
  mutual_words = []
  keywords = []
  for i in both_idx:
    if isImportantWord(i, cond_vec, med_vec, clen, mlen, word_list):
      mutual_words.append(word_list[i])

  if mutual_words:
    max_freq = sorted(mutual_words, key = lambda i: i[1], reverse = True)[0][1]
    keywords = [word for word in mutual_words if word[1]==max_freq]
  if (" " + cond.lower() + " ") in med_descr[0] and cond.lower() not in mutual_words:
    mutual_words.append(cond.lower())
    keywords.append(cond.lower())

  return keywords, mutual_words


In [ ]:
@anvil.server.callable
def get_med_info(med, all_conditions, patient_id=190539, df=df):
  
  conditions = all_conditions
  drug_names, brand_names, med_descr, norm_sides, severe_sides = get_med_description(med, True)

  try:
    med_descr = med_descr.decode('UTF-8')
  except:
    med_descr = med_descr

  key_conditions = dict()
  for cond in conditions:
    cond_descr = get_full_desc(cond)
    try:
      cond_descr = cond_descr.decode('UTF-8')
    except:
      cond_descr = cond_descr
    keywords, mutual_words = get_mutual_words(cond, med, str(cond_descr), str(med_descr))
    if mutual_words:
      key_conditions[cond] = mutual_words
      
  return med,med_descr,drug_names,brand_names,norm_sides,severe_sides, key_conditions

In [ ]:
# Anvil functionality/user flow

# 1) enter patient id (hadm_id)?
#     -> get_pt_medications() and display all medications
#     -> get_pt_conditions() and display all conditions
#     Note: each med should be a clickable button that can change color
#     Note: each condition should be text that can be highlighted on an event

# 2) select radio medication button
#     -> get_med_info() to get MedInfo object for the med
#     -> populate the cells with descriptions, side effects, etc
#     -> choose which conditions to highlight and change the formatting
#     -> REFRESH/EXPLAIN button click to get global



In [ ]:
from tqdm import tqdm

In [ ]:
def test_patient_medications(df):
  all_ids = df.hadm_id
  total = len(all_ids)
  count = 0
  for id in tqdm(all_ids):
    res = get_pt_medications(id, df)
    print(id, ":", res)
    if res:
      count += 1
  print(count, " out of ", total, " found")
    
test_patient_medications(df[:150])

In [ ]:
def test_patient_conditions(df):
  all_ids = df.hadm_id.unique()
  total = len(all_ids)
  count = 0
  for id in tqdm(all_ids):
    res = get_pt_conditions(id, df)
    print(id, ":", res)
    if res:
      count += 1
  print(count, " out of ", total, " found")

test_patient_conditions(df[:150])